In [1]:
import os
from glob import glob

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
SIZE = (50, 50)

def extract_hand(path):
   # read image as grayscale and resize it
    img = cv2.imread(path, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, SIZE)

    # blur
    img = cv2.GaussianBlur(img,(5,5),0)

    # OTSU threshold
    ret, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    return img.flatten()

In [3]:
images = []
targets = []

for number in tqdm(os.listdir('data')):
    files = glob(os.path.join('data', number, '*.png'))[:200]
    images.extend(map(extract_hand, files))
    targets.extend([number] * len(files))

100%|██████████| 6/6 [00:00<00:00, 23.02it/s]


In [4]:
x_train, x_test, y_train, y_test = train_test_split(images, targets, test_size = 0.2, random_state = 42)

model = LinearSVC()
model.fit(x_train, y_train)
pred = model.predict(x_test)

In [7]:
conf = model.decision_function(x_test)

In [5]:
accuracy_score(y_test, pred)

0.9791666666666666

In [6]:
confusion_matrix(y_test, pred)

array([[45,  0,  0,  0,  0,  0],
       [ 0, 34,  1,  0,  0,  0],
       [ 0,  3, 32,  0,  0,  0],
       [ 0,  0,  1, 47,  0,  0],
       [ 0,  0,  0,  0, 38,  0],
       [ 0,  0,  0,  0,  0, 39]])